- https://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html

# WORD EMBEDDINGS: ENCODING LEXICAL SEMANTICS

## Word Embeddings in Pytorch 

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [8]:
word_to_ix = {"hello": 0, "world": 1}
embeds = nn.Embedding(2, 5)  # 2 words in vocab, 5 dimensional embeddings

for k,v in word_to_ix.items():
    print(k,v)

    lookup_tensor = torch.tensor([word_to_ix[k]], dtype=torch.long)
    embed = embeds(lookup_tensor)
    print(embed)

hello 0
tensor([[-1.0508, -0.2513, -0.7588, -0.9832,  0.2355]],
       grad_fn=<EmbeddingBackward>)
world 1
tensor([[ 1.4564,  0.0823, -0.1229, -0.4419,  2.2642]],
       grad_fn=<EmbeddingBackward>)


## An Example: N-Gram Language Modeling

In [9]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10

In [10]:
# We will use Shakespeare Sonnet 2
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()
# we should tokenize the input, but we will ignore that for now
# build a list of tuples.  Each tuple is ([ word_i-2, word_i-1 ], target word)
trigrams = [([test_sentence[i], test_sentence[i + 1]], test_sentence[i + 2])
            for i in range(len(test_sentence) - 2)]
# print the first 3, just so you can see what they look like
print(trigrams[:3])

[(['When', 'forty'], 'winters'), (['forty', 'winters'], 'shall'), (['winters', 'shall'], 'besiege')]


In [12]:
vocab = set(test_sentence)
word_to_ix = {word: i for i, word in enumerate(vocab)}

print(vocab)
print()
print(word_to_ix)

{'praise', "totter'd", 'use,', 'besiege', 'shall', 'make', 'all-eating', 'be', 'trenches', 'sunken', 'mine', 'Where', 'Thy', 'thou', 'small', 'answer', 'couldst', 'of', 'to', 'dig', 'thine!', 'Shall', 'my', 'days;', 'shame,', 'within', 'gazed', 'Then', 'eyes,', 'livery', 'old', 'thine', 'This', 'weed', 'old,', 'blood', 'Were', 'winters', 'thy', 'where', 'count,', 'Proving', 'art', 'To', 'more', 'beauty', 'it', 'new', 'cold.', 'say,', 'see', 'on', 'when', 'thriftless', 'brow,', 'proud', 'warm', 'field,', 'so', "beauty's", 'lusty', 'held:', 'When', 'forty', 'in', "deserv'd", 'by', 'Will', 'praise.', 'lies,', 'And', 'much', 'a', 'child', 'were', 'being', 'fair', "'This", 'the', 'now,', 'and', 'deep', 'an', "feel'st", 'asked,', "youth's", 'If', 'treasure', 'succession', "excuse,'", 'worth', 'sum', 'made', 'all', 'How', 'his', 'own'}

{'praise': 0, "totter'd": 1, 'use,': 2, 'besiege': 3, 'shall': 4, 'make': 5, 'all-eating': 6, 'be': 7, 'trenches': 8, 'sunken': 9, 'mine': 10, 'Where': 11, 'T

In [15]:
class NGramLanguageModeler(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs


losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

for epoch in range(10):
    total_loss = 0
    for context, target in trigrams:
        
        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in tensors)
        context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)
        
        # Step 2. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        model.zero_grad()

        # Step 3. Run the forward pass, getting log probabilities over next
        # words
        log_probs = model(context_idxs)

        # Step 4. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a tensor)
        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    losses.append(total_loss)
print(losses)  # The loss decreased every iteration over the training data!

[526.534695148468, 523.9709868431091, 521.4293613433838, 518.9080128669739, 516.4064037799835, 513.9233894348145, 511.458838224411, 509.0114324092865, 506.58036398887634, 504.16339921951294]


## Exercise: Computing Word Embeddings: Continuous Bag-of-Words

In [16]:
CONTEXT_SIZE = 2  # 2 words to the left, 2 to the right
raw_text = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells.""".split()

# By deriving a set from `raw_text`, we deduplicate the array
vocab = set(raw_text)
vocab_size = len(vocab)

word_to_ix = {word: i for i, word in enumerate(vocab)}
data = []
for i in range(2, len(raw_text) - 2):
    context = [raw_text[i - 2], raw_text[i - 1],
               raw_text[i + 1], raw_text[i + 2]]
    target = raw_text[i]
    data.append((context, target))
print(data[:5])


class CBOW(nn.Module):

    def __init__(self):
        pass

    def forward(self, inputs):
        pass

# create your model and train.  here are some functions to help you make
# the data ready for use by your module


def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)


make_context_vector(data[0][0], word_to_ix)  # example

[(['We', 'are', 'to', 'study'], 'about'), (['are', 'about', 'study', 'the'], 'to'), (['about', 'to', 'the', 'idea'], 'study'), (['to', 'study', 'idea', 'of'], 'the'), (['study', 'the', 'of', 'a'], 'idea')]


tensor([12, 31, 39, 37])